# Libraries

In [ ]:
import pandas as pd
import nltk

#Downloading Arabic Stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv('fake_news_data.csv')
data.head()

,Label,Topic,Article_content
0,real,politics,القدس المحتلة- لن يتمكن المقدسي سامي درويش في ...
1,real,politics,طهران- منذ الهجوم الإسرائيلي على القنصلية الإي...
2,real,politics,غادر المحامي الفلسطيني راجي صوراني قطاع غزة رف...
3,real,politics,نابلس- لليوم الثاني على التوالي، تتعرض قرية دو...
4,real,politics,بريتوريا- تقدمت اللجنة المستقلة للانتخابات في ...


#Cleaning the data

In [ ]:
import re

def Cleaning_text(text):

    # Make the text only letters
    text = re.sub(r'[^أ-يa-zA-Z\s]', '', text)

    # If the text has more than 1 space between 2 words i will make it only 1
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove Arabic stopwords
    arabic_stopwords = set(stopwords.words('arabic'))
    cleaned_words = []
    for word in text.split():
      if word not in arabic_stopwords:
        cleaned_words.append(word)
        text = ' '.join(cleaned_words)

    return text

In [ ]:
data['cleaned_article'] = data['Article_content'].apply(Cleaning_text)
data.head()

,Label,Topic,Article_content,cleaned_article
0,real,politics,القدس المحتلة- لن يتمكن المقدسي سامي درويش في ...,القدس المحتلة يتمكن المقدسي سامي درويش موسم ال...
1,real,politics,طهران- منذ الهجوم الإسرائيلي على القنصلية الإي...,طهران الهجوم الإسرائيلي القنصلية الإيرانية سور...
2,real,politics,غادر المحامي الفلسطيني راجي صوراني قطاع غزة رف...,المحامي الفلسطيني راجي صوراني قطاع غزة رفقة عا...
3,real,politics,نابلس- لليوم الثاني على التوالي، تتعرض قرية دو...,نابلس لليوم الثاني التوالي تتعرض قرية دوما جنو...
4,real,politics,بريتوريا- تقدمت اللجنة المستقلة للانتخابات في ...,بريتوريا تقدمت اللجنة المستقلة للانتخابات جنوب...


# Splitting

### Split Feature and Target

In [ ]:
X = data['cleaned_article']
y = data['Label']

### Split Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature Selection for TF-IDF

### Optimal Number of Features


In [ ]:
unique_words = set(" ".join(X_train).split())
optimal_max_features = min(5000, len(unique_words))

### Transform Text into Features


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=optimal_max_features)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Logistic Regression

### Training

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()
logistic_model.fit(X_train_tfidf, y_train)

LogisticRegression()

### Testing

In [ ]:
log_y_hat = logistic_model.predict(X_test_tfidf)

### Result

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
print("Accuracy:", accuracy_score(y_test, log_y_hat))
print("Classification Report:\n", classification_report(y_test, log_y_hat))

Accuracy: 0.9910339840925524
Classification Report:
               precision    recall  f1-score   support

        fake       0.99      1.00      1.00     12641
        real       0.98      0.91      0.95      1189

    accuracy                           0.99     13830
   macro avg       0.99      0.96      0.97     13830
weighted avg       0.99      0.99      0.99     13830



# Naive Bayes

### Training

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)

MultinomialNB()

### Testing

In [ ]:
nb_y_hat = nb_model.predict(X_test_tfidf)

### Result

In [ ]:
print("Accuracy:", accuracy_score(y_test, nb_y_hat))
print("Classification Report:\n", classification_report(y_test, nb_y_hat))

Accuracy: 0.9820679681851049
Classification Report:
               precision    recall  f1-score   support

        fake       0.99      0.99      0.99     12641
        real       0.89      0.90      0.90      1189

    accuracy                           0.98     13830
   macro avg       0.94      0.95      0.94     13830
weighted avg       0.98      0.98      0.98     13830

